# Reinforcement Learning Project
#### by Luca Subitoni and Massimiliano Nigro

Code relative to the "reproducibility challenge" project of the <a href="https://www11.ceda.polimi.it/manifestidott/manifestidott/controller/MainPublic.do?EVN_DETTAGLIOINSEGNAMENTO=EVENTO&c_insegn=061642&aa=2023&k_corso_la=1380">Reinforcement Learning PhD course</a> offered by Politecnico di Milano.

Selected paper: <a href="https://arxiv.org/abs/1606.03476">Generative Adversarial Imitation Learning</a> by Jonathan Ho and Stefano Ermon.

#### CODE DESCRIPTION:
The code is meant to work on Google Colab (last check: 28th June 2024). 
The setup of the Jupyter Notebook is subdivided in the following sections:

- <b>Install packages</b>: it installs the necessary packages (including Mujoco) and loads the RL_project folder (which you can download in the GitHub repository) from your Google Drive account. A script from the imitation Pythoin library is corrected

- <b>General functions</b>: definition of Python functions useful throughout the code

The main code is then subdivided in the following sections:

1. <b>Creating the environment for both the expert policy and the imitation algorithms</b>: creations of the environments (must run)
2. <b>Train the expert algorithm</b>: uses TRPO to train an expert policy and a random policy (optional, if you already have run this, it can be skipped! Load the previous results in the Google Colab file manager and go to number 3.)
3. <b>Imitation learning</b>: defines the imitation learning algorithm implemented, i.e., Behavioral Cloning and GAIL. (must run)
4. <b>Compare the GAIL and Behavioral Cloning to the expert</b>: train and compare the imitation algorithms to the expert and the random policy (must run)
5. <b>Plot curves to visualize the comparison</b>: generates the normalized curves present in the GAIL paper. If you already have the .csv results, you can directly load them in the Google Colab file manager


# Install packages

In [1]:
!pip install stable-baselines3[extra] # Does not have TRPO
!pip install sb3-contrib # For TRPO
!pip install imitation # Imitation Learning
!pip install renderlab  # For rendering environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.

In [2]:
# MuJoCo Setup (part 1)
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install free-mujoco-py

# MuJoCo Setup (part 2)
import os
if not os.path.exists('.mujoco_setup_complete'):
  # Get the prereqs
  !apt-get -qq update
  !apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  # Get Mujoco
  !mkdir ~/.mujoco
  !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O mujoco.tar.gz
  !tar -zxf mujoco.tar.gz -C "$HOME/.mujoco"
  !rm mujoco.tar.gz
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  !echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/.mujoco/mujoco210/bin' >> ~/.bashrc
  !echo 'export LD_PRELOAD=$LD_PRELOAD:/usr/lib/x86_64-linux-gnu/libGLEW.so' >> ~/.bashrc
  # THE ANNOYING ONE, FORCE IT INTO LDCONFIG SO WE ACTUALLY GET ACCESS TO IT THIS SESSION
  !echo "/root/.mujoco/mujoco210/bin" > /etc/ld.so.conf.d/mujoco_ld_lib_path.conf
  !ldconfig
  # Install Mujoco-py
  !pip3 install -U 'mujoco-py<2.2,>=2.1'
  # run once
  !touch .mujoco_setup_complete

try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  try:
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/root/.mujoco/mujoco210/bin'
  except KeyError:
    os.environ['LD_LIBRARY_PATH']='/root/.mujoco/mujoco210/bin'
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  import mujoco_py
  _mujoco_run_once = True

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
The following additional packages will be installed:
  libegl-dev libgl-dev libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev
  libglvnd-dev libglx-dev libopengl-dev libosmesa6
The following NEW packages will be installed:
  libegl-dev libgl-dev libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6
  libosmesa6-dev
0 upgraded, 15 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,020 kB of archives.
After this operation, 19.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-31

In [3]:
# Changing a function inside the package "imitation" in the script "util.py"
# See issue: https://github.com/HumanCompatibleAI/imitation/issues/820
from google.colab import drive
drive.mount('/content/drive')
!cp -r "/content/drive/My Drive/RL_project/util.py" "/usr/local/lib/python3.10/dist-packages/imitation/util/"

Mounted at /content/drive


# General functions

In [4]:
def evaluate(env, policy, gamma=1., num_episodes=50, deterministic=True, print_results=False, use_random_policy=False):
    """
    DESCRIPTION:
        Evaluate a RL agent
    INPUT:
        - env (Env object): the Gym environment
        - policy (BasePolicy object): the policy in stable_baselines3
        - gamma (float, default=1.0): the discount factor
        - num_episodes (int, default=50): number of episodes to evaluate it
        - deterministic (bool, default=True): whether to use a deterministic policy or not
        - print_results (bool, default=False): whether to print results or not
        - use_random_policy (bool, default=False): whether to use a random policy or not (useful since it is the reference used in the GAIL paper to compare the algorithms)
    OUTPUT:
        - mean_episode_reward (float): the mean rewards across the "num_episodes" episodes
        - std_episode_reward: the standard deviation across the tested "num_episodes" episodes
    """

    from tqdm import tqdm
    import numpy as np
    
    all_episode_rewards = []
    for i in tqdm(range(num_episodes)): # iterate over the episodes
        episode_rewards = []
        done = False
        discounter = 1.
        obs, _ = env.reset()
        while not done: # iterate over the steps until termination

            # If I want to use a random policy I sample a random action
            if use_random_policy == True:
                action = env.action_space.sample()

            # otherwise I use the model to predict the action to take
            else:
                action, _ = policy.predict(obs, deterministic=deterministic)

            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_rewards.append(reward * discounter) # compute discounted reward
            discounter *= gamma

        all_episode_rewards.append(sum(episode_rewards))

    if print_results:
        mean_episode_reward = np.mean(all_episode_rewards)
        std_episode_reward = np.std(all_episode_rewards) / np.sqrt(num_episodes - 1)
        print("\nMean reward:", mean_episode_reward,
              "- Std reward:", std_episode_reward,
              "- Num episodes:", num_episodes)

        return mean_episode_reward, std_episode_reward

In [5]:
def plot_train_curve(log_dir):

    """
    DESCRIPTION:
        This function is used to plot the train curve of the experts
    INPUT:
        - log_dir (string): the string which specifies the path which contains the "evaluations.npz" file
    OUTPUT:
        - None: it plots and saves (in the log_dir path) the train curve figure
    """

    import numpy as np
    import matplotlib.pyplot as plt

    data = np.load(log_dir+"evaluations.npz")

    timesteps = data["timesteps"] # n_timesteps
    rewards = data["results"] # n_timesteps, n_eval_episodes

    plt.figure(figsize=(12,4))
    plt.plot(timesteps, rewards, color="black", alpha=0.1, lw=2)
    plt.plot(timesteps, np.mean(rewards, axis=1), color="red", lw=2, label="Median reward")
    plt.xlabel("Timesteps")
    plt.ylabel("Reward")
    plt.grid()
    plt.savefig(log_dir+"train_reward_curve.png", dpi=300)
    plt.show()

In [6]:
# General imports
import numpy as np
import torch as th
import os, shutil
import pandas as pd
import matplotlib.pyplot as plt

# To download iteratively the files from Google Colab
from google.colab import files

# Functions for the RL expert policy
import gymnasium as gym
from sb3_contrib import TRPO
import renderlab

# Function to load the expert policy for the imitation package
from imitation.policies.serialize import load_policy

# Functions for creating the vectorized environments
from imitation.util import util
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

# Importing the function which creates the rollouts for the imitation algorithm
from imitation.data import rollout

# Functions for defining custom policies
from stable_baselines3.common import policies, torch_layers

# Behavioral cloning
from imitation.algorithms import bc

# Callbacks to stop the training the expert
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, CallbackList
from stable_baselines3.common.monitor import Monitor

# Removing the sample_data folder from Google Colab
if os.path.exists("/content/sample_data/"):
    shutil.rmtree("/content/sample_data/")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
  np.bool8: (False, True),

  from scipy.ndimage.filters import sobel



# Code

## 1. Creating the environment for both the expert policy and the imitation algorithms

In [7]:
# List of environments to choose from (CartPole and Acrobot are slightly different from the ones used in the paper, but it should not be a problem in a meaningful way)
env_list_string = ["CartPole-v1", # Classic 0 (v0 is deprecated) --> Increased max_episode_steps (from 200 to 500) + increased reward_threshold (from 195.0 to 475.0)
                   "Acrobot-v1", # Classic 1 (v0 is deprecated) --> Instead of the direct angles, it provides the sine and cosine of each angle + reward rescaled from +-200 to +-500
                   "MountainCar-v0", # Classic 2 (ok)
                   "HalfCheetah-v2", # MuJoCo 3 (v1 is deprecated) --> No differences, only provides compatibility with newer MuJoCo simulator
                   "Hopper-v2", # MuJoCo 4 (v1 is deprecated) --> No differences, only provides compatibility with newer MuJoCo simulator
                   "Walker2d-v2", # MuJoCo 5 (v1 is deprecated) --> No differences, only provides compatibility with newer MuJoCo simulator
                   "Ant-v2", # MuJoCo 6 (v1 is deprecated) --> No differences, only provides compatibility with newer MuJoCo simulator
                   "Humanoid-v2", # MuJoCo 7 (v1 is deprecated) --> No differences, only provides compatibility with newer MuJoCo simulator
                   "Reacher-v2"] # MuJoCo 8 (v1 is deprecated) --> No differences, only provides compatibility with newer MuJoCo simulator

# Queste soglie, tranne la prima, sono quelle del paper, volendo si possono abbassare un po' per evitare tempi di calcolo enormi
# Thresholds for rewards (i.e. when I consider the model an expert)
reward_thr = {"CartPole-v1": 500.00, # Different from the paper since they changed the maximum_episode_steps from v0 to v1
              "Acrobot-v1": -75.25,
              "MountainCar-v0": -98.75,
              "Reacher-v2": -4.09,
              "HalfCheetah-v2": 4463.46,
              "Hopper-v2": 3571.38,
              "Walker2d-v2": 6717.08,
              "Ant-v2": 4228.37,
              "Humanoid-v2": 9575.40,
              }

# String ID for the environment
environment_string = env_list_string[4]
print("The selected environment is:", environment_string)

# Seed for the random number generator
seed = 42

# Number of vectorized environments (multiprocessing) --> 32 o 64 sono ok per la GPU di Colab (circa dalle 3k alle 6k iter/s)
if th.cuda.is_available():
    n_env = 64 # GPU
else:
    n_env = 8 # CPU

# Random Number Generator
rng = np.random.default_rng(seed)

# Create log directory for the current environment
log_dir = "/content/log/"+environment_string+"/"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir) # delete the old directory, if it exists
os.makedirs(log_dir)

# This is the name of the zip file which will be automatically downloaded and which will contain the results of the expert and of the random policy
results_zip_name = "results_" + environment_string + ".zip"

# Creating the vectorized environment for training the models faster
venv = util.make_vec_env(environment_string,
                         n_envs=n_env,
                         rng=rng,
                         post_wrappers=[lambda venv, _: RolloutInfoWrapper(venv)]  # needed for computing rollouts later (capire se mettere env o venv, vedi: https://imitation.readthedocs.io/en/latest/tutorials/1_train_bc.html)
                         )

# Creating the environment for evaluating the model
env_eval = gym.make(environment_string)
env_eval = Monitor(env_eval)

# Creating the environment for playing the video with renderlab and save them into the log directory
env_movie = gym.make(environment_string, render_mode = "rgb_array")
env_movie = renderlab.RenderFrame(env_movie, log_dir)

# Create a flag that tell whether I have trained the model now or whether I reloaded the zip file of the model on Google Colab
model_trained_now = False # This must not be changed (the code automatically accounts for it)

  logger.deprecation(

  logger.deprecation(



The selected environment is: Hopper-v2


## 2. Train the expert algorithm

In [ ]:
# Custom MLP policy
policy_kwargs = dict(activation_fn=th.nn.Tanh,
                     net_arch=dict(pi=[100, 100], vf=[100, 100]))

# Callbacks for stopping training when the model reaches the predefined reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=reward_thr[environment_string], verbose=1)
stop_callback = EvalCallback(env_eval, callback_on_new_best=callback_on_best, verbose=1, deterministic=True)

# Maximum training time steps (approximately, the maximum number of steps to train the algorithm)
max_time_steps = 5e6

# How often do I want to evaluate the algorithms to obtain the training curve (it is scaled by the n_envs during training)
# 10000 for classic 50000 for mujoco in eval_iter for eval_freq below! (otherwise too slow to train since it is always evaluating...)
eval_iter = {"CartPole-v1": 25000,
             "Acrobot-v1": 25000,
             "MountainCar-v0": 25000,
             "Reacher-v2": 50000,
             "HalfCheetah-v2": 50000,
             "Hopper-v2": 50000,
             "Walker2d-v2": 50000,
             "Ant-v2": 50000,
             "Humanoid-v2": 50000,
             }

# Callback for evaluating the model periodically (training curve)
eval_callback = EvalCallback(env_eval, log_path=log_dir, eval_freq=int(eval_iter[environment_string]/n_env), n_eval_episodes=10, deterministic=True, render=False)
# Create the callback list
callback = CallbackList([stop_callback, eval_callback])

# Train (log_interval è il numero di episodi prima di generare un report)
model = TRPO("MlpPolicy", venv, verbose=1, policy_kwargs=policy_kwargs)
#model.learn(total_timesteps=10_000, log_interval=1, progress_bar=True) # Train for fixed number of timesteps (utile per avere la progress bar e valutare la velocità)
# (N.B il log_interval genera un box con rollout usando una policy non deterministica e quindi si vedono risultati peggiori di quelli che sono in realtà)
model.learn(total_timesteps=int(max_time_steps), log_interval=1, callback=callback) # Train until a mean reward is reached or up until total_timesteps is reached

# Save the model
model.save(log_dir+"/"+environment_string+"_model") # Genera un file zip dove viene salvato il modello

# Plot and save the training curve
plot_train_curve(log_dir)

# Set the flag which indicates that I have trained the model now to True
model_trained_now = True

In [ ]:
# EXPERT ========================================================
# Evaluate the trained model and save the final results
model_eval_mean, model_eval_std = evaluate(env_eval, model, gamma=1., num_episodes=50, deterministic=True, print_results=True)
np.save(log_dir+"expert_evaluation.npy", np.array([model_eval_mean, model_eval_std]))

# Show movie
print("\n\n===============================")
evaluate(env_movie, model, num_episodes=1, deterministic=True, print_results=False)
env_movie.play()

In [ ]:
# RANDOM POLICY ==================================================
# Evaluate a random policy and save the final results
model_eval_mean, model_eval_std = evaluate(env_eval, policy=None, gamma=1., num_episodes=50, deterministic=True, print_results=True, use_random_policy=True)
np.save(log_dir+"random_evaluation.npy", np.array([model_eval_mean, model_eval_std]))

# Show movie
print("\n\n===============================")
evaluate(env_movie, policy=None, num_episodes=1, deterministic=True, print_results=False, use_random_policy=True)
env_movie.play()

In [ ]:
# Create a zip file to download all files
!zip -r {results_zip_name} {log_dir}
# Download the folder in which all data have been saved
files.download(results_zip_name)

## 3. Imitation learning

In [17]:
import os
import shutil

# If I have loaded the trained model on Google Colab I have to load back the results
if model_trained_now == False:

    # Unzip the zip file which containes the training results of the expert and of the random policy
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir) # delete the old directory, if it exists
    os.makedirs(log_dir)
    path_in_which_files_are_unzipped = log_dir+log_dir[1:] # Due to the zip file construction is unzipped in a subfolder
    if os.path.exists(path_in_which_files_are_unzipped):
        shutil.rmtree(path_in_which_files_are_unzipped) # delete the old directory, if it exists
    !unzip {results_zip_name} -d {log_dir}
    # Mode files in the correct log dir
    for file in os.listdir(path_in_which_files_are_unzipped):
        shutil.move(path_in_which_files_are_unzipped+file, log_dir)
    # Remove the original folder
    shutil.rmtree(log_dir+"content")
    print("The zipped model results have been loaded!")

# Testing if the model loading was correct
test_if_correct = False
if test_if_correct:
    # To load policy from disk use "ppo" as key register (https://imitation.readthedocs.io/en/latest/tutorials/1_train_bc.html)
    expert = load_policy("ppo", venv, path="/content/log/"+environment_string+"/"+environment_string+"_model")
    # Evaluate the trained model
    _, _ = evaluate(env_eval, model, gamma=1., num_episodes=10, deterministic=True, print_results=True) # Same expert model as before (you must not restart the kernel, otherwise you will not have the "model" variable)
    _, _ = evaluate(env_eval, expert, gamma=1., num_episodes=10, deterministic=True, print_results=True) # Loaded expert model

Archive:  results_Hopper-v2.zip
   creating: /content/log/Hopper-v2/content/
  inflating: /content/log/Hopper-v2/__MACOSX/._content  
  inflating: /content/log/Hopper-v2/content/.DS_Store  
  inflating: /content/log/Hopper-v2/__MACOSX/content/._.DS_Store  
   creating: /content/log/Hopper-v2/content/log/
  inflating: /content/log/Hopper-v2/__MACOSX/content/._log  
  inflating: /content/log/Hopper-v2/content/log/.DS_Store  
  inflating: /content/log/Hopper-v2/__MACOSX/content/log/._.DS_Store  
   creating: /content/log/Hopper-v2/content/log/Hopper-v2/
  inflating: /content/log/Hopper-v2/__MACOSX/content/log/._Hopper-v2  
  inflating: /content/log/Hopper-v2/content/log/Hopper-v2/Hopper-v2_model.zip  
  inflating: /content/log/Hopper-v2/__MACOSX/content/log/Hopper-v2/._Hopper-v2_model.zip  
  inflating: /content/log/Hopper-v2/content/log/Hopper-v2/expert_evaluation.npy  
  inflating: /content/log/Hopper-v2/__MACOSX/content/log/Hopper-v2/._expert_evaluation.npy  
  inflating: /content/log/

In [18]:
def get_transitions(min_timesteps=50, min_episodes=10, rng=rng, expert=None, keep_only_min_episodes=True, keep_only_500_pairs=True):

    # min_timesteps is the Number of state action pairs
    # min_episodes is the Number of trajectories in the dataset (x axis of the GAIL paper)
    # https://imitation.readthedocs.io/en/latest/_api/imitation.data.rollout.html

    from imitation.data.types import TrajectoryWithRew

    # I want the expert to behave deterministically
    kwargs = {"deterministic_policy": True}

    # Generate trajectories for the imitation learning algorithm
    rollouts = rollout.rollout(expert,
                               venv,
                               rollout.make_sample_until(min_timesteps=min_timesteps, min_episodes=min_episodes),
                               rng=rng,
                               **kwargs)

    # Keep only the first min_episodes number of trajectories (the function above can oversample sometimes)
    if keep_only_min_episodes == True:
        rollouts = rollouts[:int(min_episodes)]

    # Keep only the first 50 samples (?) (as it written in the paper --> approximately 50 pairs of observation and action for each trajectory)
    if keep_only_500_pairs == True:
        for i in range(len(rollouts)):
            rollouts[i] = TrajectoryWithRew(obs = rollouts[i].obs[:499+1], # We have one more observation than actions/rewards
                                            acts = rollouts[i].acts[:499],
                                            infos = rollouts[i].infos,
                                            terminal = rollouts[i].terminal,
                                            rews = rollouts[i].rews[:499])

    # Flatten the trajectories
    transitions = rollout.flatten_trajectories(rollouts)
    return transitions, rollouts

In [19]:
def get_obs_act(transitions):
    """
    DESCRIPTION:
        Function to get observations and actions within the transitions which were generated. 
        This function is useful for the loss function computation for the Behavioral Cloning algorithm
    INPUT:
        - transitions: the transitions of the expert, as generated by the get_transitions() function
    OUTPUT:
        - observations: the observations within the transitions
        - actions: the actions within the transitions
    """
    
    observations = np.array([transitions[i]["obs"] for i in range(len(transitions))])
    actions = np.array([transitions[i]["acts"] for i in range(len(transitions))])

    return th.from_numpy(observations), th.from_numpy(actions)

#### 3.1. Behavioral Cloning function definition

In [20]:
# Creating a custom policy as defined in the paper
class FeedForward100Policy(policies.ActorCriticPolicy):
    """
    A feed forward policy network with two hidden layers of 100 units and Tanh activation function in between.
    """

    def __init__(self, *args, **kwargs):
        """Builds FeedForward32Policy; arguments passed to `ActorCriticPolicy`."""
        super().__init__(*args, **kwargs, net_arch=[100, 100]) # Default is Tanh (see https://stable-baselines.readthedocs.io/en/master/_modules/stable_baselines/common/policies.html#FeedForwardPolicy)

In [21]:
def behavioral_cloning_train(rng=rng, play_video=False, env_eval=None, venv=None, env_movie=None, transitions=None, device="cpu"):

    """
    DESCRIPTION:
        This function performs the train of the Behavioral Cloning algorithm
    INPUT:
        - rng (default=rng, defined above in the code): random number generator
        - play_video (bool, defualt=False): whether to play the video or not
        - env_eval (default=None): the environment in which to perform the evaluation
        - venv (default=None): the vectorized environment in which to perform the training steps
        - env_movie (default=None): the fictitious environment which is used to create the movie of the expert using Google Colab
        - transition (default=None): the transitions of the expert
        - device (default="cpu"): the device used to run the Behavioral cloning training algorithm
    OUTPUT:
        - im_mean_reward: the mean reward of the Behavioral Cloning algorithm
        - im_std_reward the standard deviation of the Behavioral Cloning algorithm
    """

    from imitation.algorithms.bc import BehaviorCloningLossCalculator

    # Create the object of the policy (vedi https://imitation.readthedocs.io/en/latest/_modules/imitation/algorithms/bc.html#BC.__init__)
    custom_policy = FeedForward100Policy(observation_space=venv.observation_space,
                                        action_space=venv.action_space,
                                        lr_schedule=lambda _: th.finfo(th.float32).max,)

    # Train val split for behavioral cloning
    train_transitions = transitions[:int(len(transitions)*0.7)]
    val_transitions = transitions[int(len(transitions)*0.7):]
    loss_calculator = BehaviorCloningLossCalculator(ent_weight=0.001, l2_weight=0.0)

    # Train the imitation learning algorithm
    bc_trainer = bc.BC(observation_space=venv.observation_space,
                action_space=venv.action_space,
                demonstrations=train_transitions,
                batch_size=128,
                policy=custom_policy,
                device=device,
                rng=rng, # Questo poi va ri-inizializzato ad ogni re-run (loro fanno 5-7 re-run)
    )

    # Compute the loss in order to perform an early stopping based on the validation curve
    loss = np.inf
    current_loss = -np.inf
    while current_loss < loss:
        # If it is not the first loop
        if current_loss != -np.inf:
            loss = current_loss
        # Train the algorithm
        bc_trainer.train(n_epochs=1)
        # Evaluate the algorithm
        val_observations, val_actions = get_obs_act(val_transitions)
        current_loss = float(loss_calculator(bc_trainer.policy, val_observations, val_actions).loss)
        print(current_loss)

    # Evaluate the imitation learning algorithm
    im_mean_reward, im_std_reward = evaluate(env_eval, bc_trainer.policy, gamma=1., num_episodes=50, deterministic=True, print_results=True) # Loaded expert model

    # Play the video
    if play_video:
        evaluate(env_movie, bc_trainer.policy, num_episodes=1, deterministic=True, print_results=False)
        env_movie.play()

    return im_mean_reward, im_std_reward

#### 3.2. GAIL function definition

In [22]:
from torch import nn
from imitation.util import networks, util
from imitation.rewards.reward_nets import RewardNet
from stable_baselines3.common import preprocessing
from typing import Any, Callable, Dict, Iterable, Optional, Sequence, Tuple, Type, cast

class CustomRewardNet(RewardNet):

    def __init__(
        self,
        observation_space: gym.Space,
        action_space: gym.Space,
        use_state: bool = True,
        use_action: bool = True,
        use_next_state: bool = False,
        use_done: bool = False,
        **kwargs,
    ):
        """Builds reward MLP.

        Args:
            observation_space: The observation space.
            action_space: The action space.
            use_state: should the current state be included as an input to the MLP?
            use_action: should the current action be included as an input to the MLP?
            use_next_state: should the next state be included as an input to the MLP?
            use_done: should the "done" flag be included as an input to the MLP?
            kwargs: passed straight through to `build_mlp`.
        """
        super().__init__(observation_space, action_space)
        combined_size = 0

        self.use_state = use_state
        if self.use_state:
            combined_size += preprocessing.get_flattened_obs_dim(observation_space)

        self.use_action = use_action
        if self.use_action:
            combined_size += preprocessing.get_flattened_obs_dim(action_space)

        self.use_next_state = use_next_state
        if self.use_next_state:
            combined_size += preprocessing.get_flattened_obs_dim(observation_space)

        self.use_done = use_done
        if self.use_done:
            combined_size += 1

        full_build_mlp_kwargs: Dict[str, Any] = {
            "hid_sizes": (100, 100),
            "activation": th.nn.Tanh,
            **kwargs,
            # we do not want the values below to be overridden
            "in_size": combined_size,
            "out_size": 1,
            "squeeze_output": True,
        }

        self.mlp = networks.build_mlp(**full_build_mlp_kwargs)


    def forward(self, state, action, next_state, done):
        inputs = []
        if self.use_state:
            inputs.append(th.flatten(state, 1))
        if self.use_action:
            inputs.append(th.flatten(action, 1))
        if self.use_next_state:
            inputs.append(th.flatten(next_state, 1))
        if self.use_done:
            inputs.append(th.reshape(done, [-1, 1]))

        inputs_concat = th.cat(inputs, dim=1)

        outputs = self.mlp(inputs_concat)
        assert outputs.shape == state.shape[:1]

        return outputs


In [23]:
def gail_train(training_iterations=300, state_action_pairs_per_iteration=5000, play_video=False, env_eval=None,  venv=None, env_movie=None, rollouts=None):
    """
    DESCRIPTION:
        This function performs the train of the GAIL algorithm
    INPUT:
        - training_iterations (int, default=300): the number of training iterations of the GAIL algorithm (as described in the Appendix of the paper)
        - state_action_pairs_per_iteration (int, default=5000): the number of state action pairs per iteration (as described in the Appendix of the paper)
        - play_video (bool, defualt=False): whether to play the video or not
        - env_eval (default=None): the environment in which to perform the evaluation
        - venv (default=None): the vectorized environment in which to perform the training steps
        - env_movie (default=None): the fictitious environment which is used to create the movie of the expert using Google Colab
        - rollouts (default=None): the rollouts generated by the expert
    OUTPUT:
        - im_mean_reward: the mean reward of the GAIL algorithm
        - im_std_reward the standard deviation of the GAIL algorithm
    """

    from imitation.rewards.reward_nets import BasicRewardNet
    from stable_baselines3.ppo import MlpPolicy
    from sb3_contrib import TRPO
    from imitation.algorithms.adversarial.gail import GAIL

    # Compute the number of iterations
    n_timesteps = int(training_iterations * state_action_pairs_per_iteration)
    training_kwargs = {"gen_train_timesteps": int(state_action_pairs_per_iteration)}

    # Custom network 1
    reward_net = CustomRewardNet(
        observation_space=venv.observation_space,
        action_space=venv.action_space
    )

    # Custom network 2
    policy_kwargs = dict(activation_fn=th.nn.Tanh,
                        net_arch=dict(pi=[100, 100], vf=[100, 100]))

    # Learning algorithm
    learner = TRPO("MlpPolicy",
                    venv,
                    verbose=1,
                    gamma=0.995,
                    gae_lambda=0.97,
                    policy_kwargs=policy_kwargs)

    # Compose GAIL trainer
    gail_trainer = GAIL(demonstrations=rollouts,
                        demo_batch_size= 1024, 
                        gen_replay_buffer_capacity=512,
                        n_disc_updates_per_round=8,
                        venv=venv,
                        gen_algo=learner,
                        reward_net=reward_net,
                        allow_variable_horizon=True,
                        **training_kwargs)

    # Train the GAIL algorithm
    gail_trainer.train(n_timesteps)

    # Evaluate the imitation learning algorithm
    im_mean_reward, im_std_reward = evaluate(env_eval, gail_trainer.policy, gamma=1., num_episodes=50, deterministic=True, print_results=True) # Loaded expert model

    # Play the video
    if play_video:
        evaluate(env_movie, gail_trainer.policy, num_episodes=1, deterministic=True, print_results=False)
        env_movie.play()

    return im_mean_reward, im_std_reward

## 4. Compare the GAIL and Behavioral Cloning to the expert

In [ ]:
# To load policy from disk use "ppo" as key register (https://imitation.readthedocs.io/en/latest/tutorials/1_train_bc.html)
expert = load_policy("ppo", venv, path="/content/log/"+environment_string+"/"+environment_string+"_model")

# Testing if the model loading was correct
test_if_correct = False
if test_if_correct:
    # Evaluate the trained model
    _, _ = evaluate(env_eval, model, gamma=1., num_episodes=10, deterministic=True, print_results=True) # Same expert model as before (you must not restart the kernel, otherwise you will not have the "model" variable)
    _, _ = evaluate(env_eval, expert, gamma=1., num_episodes=10, deterministic=True, print_results=True) # Loaded expert model


# Load the expert evaluations
expert_eval = np.load(log_dir+"expert_evaluation.npy")[0]
random_eval = np.load(log_dir+"random_evaluation.npy")[0]

# Select GAIL training iterations depending on the environment (These are the correct ones)
training_iterations_gail = {"CartPole-v1": 300,
                            "Acrobot-v1": 300,
                            "MountainCar-v0": 300,
                            "Reacher-v2": 200,
                            "HalfCheetah-v2": 500,
                            "Hopper-v2": 500,
                            "Walker2d-v2": 500,
                            "Ant-v2": 500,
                            "Humanoid-v2": 1500,
                            }
# Write here a fraction if you want to reduce the number of training iterations for computational reasons (1/5th of the true ones)
retuce_training_iterations_fraction = 1/5

state_action_pairs_per_iteration_gail = {"CartPole-v1": 5000,
                                        "Acrobot-v1": 5000,
                                        "MountainCar-v0": 5000,
                                        "Reacher-v2": 50000,
                                        "HalfCheetah-v2": 50000,
                                        "Hopper-v2": 50000,
                                        "Walker2d-v2": 50000,
                                        "Ant-v2": 50000,
                                        "Humanoid-v2": 50000,
                                        }

min_episodes_list = {"CartPole-v1": [1, 4, 7, 10],
                    "Acrobot-v1": [1, 4, 7, 10],
                    "MountainCar-v0": [1, 4, 7, 10],
                    "Reacher-v2": [4, 11, 18],
                    "HalfCheetah-v2": [4, 11, 18, 25],
                    "Hopper-v2": [4, 11, 18, 25],
                    "Walker2d-v2": [4, 11, 18, 25],
                    "Ant-v2": [4, 11, 18, 25],
                    "Humanoid-v2": [80, 160, 240],
                    }

min_timesteps = 50
mean_r_bc_list = []
mean_r_gail_list = []

# Cycling on the minimum number of trajectories (this changes for each environment...)
first_run = True
for min_episodes in min_episodes_list[environment_string]:

    print("Number of episodes selected:", min_episodes)

    # Train the algorithms for a number of times (They "run the algorithms over 5-7 reruns", in this case we use less, otherwise Google Colab shuts down the connection)
    n_reruns = 7
    for i in range(n_reruns):

        # Set new random seed
        rng = np.random.default_rng(i)

        # Get training samples
        transitions, rollouts = get_transitions(min_timesteps=50,
                                                min_episodes=min_episodes,
                                                rng=rng,
                                                expert=expert,
                                                keep_only_min_episodes=True, # We only keep the min_episodes trajectory
                                                keep_only_500_pairs=True) # We only keep the first 500 pairs of (obs,act) for each trajectory

        # Train Behavioral Cloning and get results
        mean_r_bc, std_r_bc = behavioral_cloning_train(rng=rng, env_eval=env_eval, env_movie=env_movie, venv=venv, transitions=transitions)
        mean_r_bc_list.append(mean_r_bc)

        # Train GAIL and get results (only a percentage of the training iterations, otherwise it is too slow)
        mean_r_gail, std_r_gail = gail_train(training_iterations=int(retuce_training_iterations_fraction*training_iterations_gail[environment_string]),
                                            state_action_pairs_per_iteration = state_action_pairs_per_iteration_gail[environment_string],
                                            venv=venv, rollouts=rollouts, env_eval=env_eval, env_movie=env_movie)
        mean_r_gail_list.append(mean_r_gail)

        # Print summary at the end of the for loop
        print("========================================================")
        print("Minimum number of episodes selected:", min_episodes)
        print("ITERATION", i, "- SUMMARY:")
        print("- Expert:", expert_eval)
        print("- Random:", random_eval)
        print("- Behavioral Cloning:", mean_r_bc)
        print("- GAIL:", mean_r_gail)
        print("========================================================")

        # Rescale the results between 0 (= random) and 1 (= expert)
        rescaled_mean_r_bc = (mean_r_bc - random_eval) / (expert_eval - random_eval)
        rescaled_mean_r_gail = (mean_r_gail - random_eval) / (expert_eval - random_eval)

        # Create a dataframe and appending the rows
        row_columns = ["n_episodes", "Mean Reward BC", "Mean Reward GAIL", "Rescaled Mean Reward BC", "Rescaled Mean Reward GAIL"]
        row = [min_episodes, mean_r_bc, mean_r_gail, rescaled_mean_r_bc, rescaled_mean_r_gail]
        # Iteratively create the dataframe and save it
        if first_run == True:
            dataframe = pd.DataFrame(dict(zip(row_columns, row)), index=[0])
            first_run = False
        else:
            current_row_in_dataframe = pd.DataFrame(dict(zip(row_columns, row)), index=[0])
            dataframe = pd.concat([dataframe, current_row_in_dataframe]).reset_index(drop=True)

        # Save the pandas dataframe iteratively (overwrites)
        dataframe.to_csv(log_dir+"dataframe_results.csv")
        # Save it also on Google Drive to avoid loosing results when Google Colab stops the connection
        dataframe.to_csv("/content/drive/MyDrive/RL_project/dataframe_results_"+environment_string+".csv")
        print("CSV dataframe saved!")

# Download when finished (this works only if Google Colab does not end the connection... otherwise you will find the results on your Google Drive)
files.download(log_dir+"dataframe_results.csv")

INFO:root:Loading Stable Baselines policy for '<class 'stable_baselines3.ppo.ppo.PPO'>' from '/content/log/Hopper-v2/Hopper-v2_model'


Number of episodes selected: 25


INFO:root:Rollout stats: {'n_traj': 32, 'return_min': 3593.4563449529765, 'return_mean': 3618.016123661458, 'return_std': 16.506941145310424, 'return_max': 3646.376665965219, 'len_min': 1000, 'len_mean': 1000.0, 'len_std': 0.0, 'len_max': 1000}
0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00426 |
|    entropy        | 4.26     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 223      |
|    loss           | 3.27     |
|    neglogp        | 3.27     |
|    prob_true_act  | 0.0413   |
|    samples_so_far | 32       |
--------------------------------


266batch [00:02, 119.77batch/s]
272batch [00:02, 102.89batch/s]


...

--------------------------------------------------
| raw/                                |          |
|    gen/rollout/ep_len_mean          | 1e+03    |
|    gen/rollout/ep_rew_mean          | 3.63e+03 |
|    gen/rollout/ep_rew_wrapped_mean  | 680      |
|    gen/time/fps                     | 2320     |
|    gen/time/iterations              | 1        |
|    gen/time/time_elapsed            | 7        |
|    gen/time/total_timesteps         | 5324800  |
|    gen/train/explained_variance     | 0.935    |
|    gen/train/is_line_search_success | 1        |
|    gen/train/kl_divergence_loss     | 0.00981  |
|    gen/train/learning_rate          | 0.001    |
|    gen/train/n_updates              | 324      |
|    gen/train/policy_objective       | 0.0249   |
|    gen/train/std                    | 0.147    |
|    gen/train/value_loss             | 2.7      |
--------------------------------------------------
--------------------------------------------------
| raw/                         

round:  82%|████████▏ | 82/100 [1:27:02<19:26, 64.82s/it]

--------------------------------------------------
| raw/                                |          |
|    gen/rollout/ep_len_mean          | 999      |
|    gen/rollout/ep_rew_mean          | 3.62e+03 |
|    gen/rollout/ep_rew_wrapped_mean  | 720      |
|    gen/time/fps                     | 1995     |
|    gen/time/iterations              | 1        |
|    gen/time/time_elapsed            | 8        |
|    gen/time/total_timesteps         | 5390336  |
|    gen/train/explained_variance     | 0.966    |
|    gen/train/is_line_search_success | 1        |
|    gen/train/kl_divergence_loss     | 0.00657  |
|    gen/train/learning_rate          | 0.001    |
|    gen/train/n_updates              | 328      |
|    gen/train/policy_objective       | 0.0249   |
|    gen/train/std                    | 0.145    |
|    gen/train/value_loss             | 2.34     |
--------------------------------------------------
--------------------------------------------------
| raw/                         

round:  83%|████████▎ | 83/100 [1:28:07<18:24, 64.99s/it]

--------------------------------------------------
| raw/                                |          |
|    gen/rollout/ep_len_mean          | 999      |
|    gen/rollout/ep_rew_mean          | 3.62e+03 |
|    gen/rollout/ep_rew_wrapped_mean  | 684      |
|    gen/time/fps                     | 1795     |
|    gen/time/iterations              | 1        |
|    gen/time/time_elapsed            | 9        |
|    gen/time/total_timesteps         | 5455872  |
|    gen/train/explained_variance     | 0.969    |
|    gen/train/is_line_search_success | 1        |
|    gen/train/kl_divergence_loss     | 0.00974  |
|    gen/train/learning_rate          | 0.001    |
|    gen/train/n_updates              | 332      |
|    gen/train/policy_objective       | 0.0346   |
|    gen/train/std                    | 0.145    |
|    gen/train/value_loss             | 1.37     |
--------------------------------------------------
--------------------------------------------------
| raw/                         

## 5. Plot curves to visualize the comparison

This cell can be executed as a standalone cell if you re-upload the dataframe results which were stored in the .csv file

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# If I reuploaded the results, I have to change here the environment string of the title
if model_trained_now == False:
    environment_string = "Ant-v2"

# Loading the dataframe which contains the results of the relative environment
dataframe = pd.read_csv("dataframe_results_" + environment_string + ".csv", index_col=0)

# Group the results and compute the mean and standard deviation of the rescaled results
dataframe_grouped = dataframe.groupby("n_episodes").agg(
                    mean_rescaled_bc = ("Rescaled Mean Reward BC", "mean"),
                    std_rescaled_bc = ("Rescaled Mean Reward BC", "std"),
                    mean_rescaled_gail = ("Rescaled Mean Reward GAIL", "mean"),
                    std_rescaled_gail = ("Rescaled Mean Reward GAIL", "std"))

# PLOT
# ==============================================================================
plt.figure(figsize=(10,5))

# Expert
plt.axhline(0, linestyle="dashed", lw=2, color="tab:green", label="Random")

# Random
plt.axhline(1, linestyle="dashed", lw=2, color="tab:blue", label="Expert")

# Plot for Rescaled Mean Reward GAIL
plt.plot(dataframe_grouped.index, dataframe_grouped['mean_rescaled_gail'], label='GAIL', marker='s', color="#add8e6")
plt.fill_between(dataframe_grouped.index,
                 dataframe_grouped['mean_rescaled_gail'] - dataframe_grouped['std_rescaled_gail'],
                 dataframe_grouped['mean_rescaled_gail'] + dataframe_grouped['std_rescaled_gail'],
                 alpha=0.2,
                 color="#add8e6")

# Plot for Rescaled Mean Reward BC
plt.plot(dataframe_grouped.index, dataframe_grouped['mean_rescaled_bc'], label='Behavioral Cloning', marker='o',color="tab:orange")
plt.fill_between(dataframe_grouped.index,
                 dataframe_grouped['mean_rescaled_bc'] - dataframe_grouped['std_rescaled_bc'],
                 dataframe_grouped['mean_rescaled_bc'] + dataframe_grouped['std_rescaled_bc'],
                 alpha=0.2,
                 color="tab:orange")


plt.xlabel('Number of trajectories in dataset')
plt.ylabel('Performance (scaled)')
plt.title(environment_string)
plt.legend(loc="lower center", ncol=2)
plt.ylim([-0.3, 1.1])
plt.xticks(dataframe_grouped.index.to_list())
plt.yticks([0, 0.2, 0.4, 0.6, 0.6, 0.8, 1])
plt.grid(True)
plt.show()